# Vistória de Viatura - PRF PROJECT
Alunos:
<br>
Disciplína: Inteligência Antificial - 2022.2  

<h3>Passos a serem seguidos:</h3> 

1. Extração dos dados

2. Técnicas de pré-processamento textual

    2.1 Remoção de Stopwords <br>
    2.2 Tokenization <br>
    2.3 Stemming <br>
    2.4 Lemmatization <br>
    2.5 Pos Tags 
    2.6 Named Entity Recognition <br>
    2.7 Chunking

3. Representação textual

    3.1 One-hot encode <br>
    3.2 BagofWords <br>
    3.3 TF-IDF <br>
    3.4 Word2Vec <br>
    3.5 GloVe




 TO DO
 
 - Utilizar AltoUml (comparação);
 - <s>Alterar o argumentos das funções;</s>
 - <s>Verificar os próximos tópicos de pré-processamento textual</s>;
 - <s>Passar o texto para números para poder usar algoritmo de classificacão;</s>
 - Verificar se as bases de dados (treinamento e testes) estão balanceadas;
 -<s> Parquet (formato de arquivo)</s>
 
 - <s> Usar matriz de confusão ( verificar erros ) </s>
 - <s>Verificar balanceamento ( 1's e 0's ) ( evitar overfitting )</s>
 - Fazer integração com fast api
 - Comparar com TF- IDF
 - Listar problemas de acuracia e possibildades pro futuro (  Utilizar fuzzy para melhorar a classificação e pre agrupamento )
 - Ver alto uml
 - Slide
 - <s>Criar documentação</s>
 - <s>Ajustar os diagramas</s>
 - <s>Fazer Orientação a objetos (ultima coisa)</s>


#Para Rodar
 ## - importar um arquivo data.csv
 ## -o  arquivo precisa seguir a seguinte estrutura:
 #### index,alteracoes,target
 #### 0,sem problemas,0
 #### 1,com problemas,1
 #### 2,carro quebrado,1
 #### 3,capô arranhado,0
 #### ...

## Há um arquivo data.csv disponivel

# Codigo


<h3>Pacotes necessários necessárias para rodar o código</h3>

<h3>Importação de pacotes e download de bibliotecas</h2>

In [ ]:
#imports e loads
import io
import pandas as pd
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag
from gensim.models import Word2Vec
import numpy as np


#Pacotes necessários para aplicar o balanceamento dos dados
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from imblearn.under_sampling import RandomUnderSampler
# configurar o estilo dos gráficos com o Seaborn
sns.set_style('dark')

#Pacote para plotar a matriz de confusão
from sklearn.metrics import confusion_matrix


#Pacote utilizado na lematização
!python -m spacy download pt_core_news_lg

#pacote necessario para fazer a lematização
nlp = spacy.load('pt_core_news_lg')

#pacote necessario para fazer a remoção de stopwords
nltk.download('stopwords')

#aumento no tamanho maximo que os metodos da spacy aceitam(https://datascience.stackexchange.com/questions/38745/increasing-spacy-max-nlp-limit)
nlp.max_length = 1904483

#lista com stopwords
stopwords = stopwords.words('portuguese')

#Gerar as wordembeddings
import gensim

#Construir o modelo e avaliação

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import  f1_score

print('downloads were done')

2023-04-25 10:50:12.807987: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 2.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
downloads were done


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<h3>Pré-processamento da base de dados</h3>

<h4>Alterando formato dos parametros<h4>

In [ ]:
df = pd.read_csv('data.csv')
#Remover as linhas nas quais os elementos sejam nan
df.isnull().sum()

index            0
alteracoes       0
target        4214
dtype: int64

<h4>Formatando a base de dados</h4>

In [ ]:
new_data = df.dropna(axis = 0, how ='any') 
new_data.isnull().sum()

index         0
alteracoes    0
target        0
dtype: int64

#Balanceando os dados (Aplicando o Under-sampling)

<h4> Verificando o balanceamento das classes</h4>

In [ ]:
print(new_data.target.value_counts())
qtd_nao_urgentes = (new_data[new_data.target == 0].shape[0] / new_data.shape[0]) * 100
print(f'\nNão problema corresponde a: {qtd_nao_urgentes:.2f} da base de dados')

0.0    1773
1.0    1576
Name: target, dtype: int64

Não problema corresponde a: 52.94 da base de dados


<h4>Dividindo a base de dados de treino e teste</h4>



In [ ]:
df_treino, df_teste = train_test_split(new_data, test_size=0.09, stratify=new_data['target'])#modificar parametros
df_treino

,index,alteracoes,target
479,479,painel acusando anomalia referente ao air bag ...,1.0
7020,7020,a vtr encontra com a base da antena esquerda d...,1.0
7368,7368,viatura retornou com a equipe que estava convo...,0.0
4188,4188,01 liquido de arrefecimento do motor ok 02 flu...,0.0
6960,6960,em manutencao pane eletrica sem movimentacao,0.0
...,...,...,...
7374,7374,sem disco no tacografo,0.0
4331,4331,camara de re nao funciona,1.0
2304,2304,viatura envolvida em acidente de transito,1.0
5265,5265,pastilhas ruins desgaste excessivo dos pneus d...,1.0


In [ ]:
df_treino.groupby('target').count()

,index,alteracoes
target,,
0.0,1613,1613
1.0,1434,1434


In [ ]:
df_teste.groupby('target').count()

,index,alteracoes
target,,
0.0,160,160
1.0,142,142


 <h3>Pré-processamento textual<h3>

<h4>Tokenization<h4>

In [ ]:
def tokenization(alteracoes:str):
  """Retirar os tokens de uma texto. Os tokens podem ser pontuação e numeral.
  
  Args:
    info (string): Frase que será tokenizada.
   
  return:
    dados_tokenizados (lista): Dados tokenizados.   
  """
  #Aqui aplica-se expressões regulares(Detecção de pontuação e numeral)
  alteracoes=alteracoes.lower()
  dados_tokenizados = list()
  tokenizer = RegexpTokenizer(r'[A-z]\w*') 
  # Criar a variável com os tokens sem pontuação e numeral oriundos do texto.
  dados_tokenizados = tokenizer.tokenize(alteracoes)
  return dados_tokenizados

<h4>Remoção de stopwords</h4>

In [ ]:
def removeStopwords(tokens:list()):
  """Remoção de stopwords, ou seja, remoção de palavras que podem ser consideradas irrelevantes para algumas análises, como artigos e preposições).
  
  Args:
    tokens (lista): Dados tokenizados.

  return:
    dados_sem_stopwords (lista): Dados com as stopwords removidas.
  
  """
  #Lista com palavras que não devem serem removidas
  exception_list = ["sem"]
  # Criação da variável contendo os tokens sem stopwords
  dados_sem_stopword = [word for word in tokens if word not in stopwords or word in exception_list]
  return dados_sem_stopword

<h4>Lematization</h4>

In [ ]:
def lematization(tokens_palavras:list()):
  """ Reduz as palavras de acordo com seus componentes lexicais raiz, ou seja, reduz a palavra até o seu lemma (palavra base) 
  
  Args:
    tokens_palavras (lista): Dados com as stopwords removidas.
  return:
    dados_lematizados (lista): Dados lematizados.
  """

  #Lista de palavras com a lematização. 
  dados_lematizados = list()
  #Unindo os dados sem stopwords em uma frase.
  frase = ' '.join(tokens_palavras)
  doc = nlp(frase)
  dados_lematizados = [token.lemma_ for token in doc]
  return dados_lematizados

<h4>Aplicando o pré-processamento nos dados de treino e teste</h4>

In [ ]:
def pre_processamento(df):
  df['dados_tokenizados'] = None 
  df['dados_sem_stopwords'] = None
  df['dados_lematizados'] = None

  #Interando sobre os dados e realizando o pré-processamento
  for index, value in df.iterrows():
    #Aplicando a tokenização
    tokenizacao = tokenization(value['alteracoes'])
    #Armazenando os dados tokenizados na base de dados
    df.at[index,'dados_tokenizados'] = tokenizacao
    #Aplicando a remoção de stopwords
    sem_stopwords = removeStopwords(tokenizacao)
    #Armazenando os dados sem stopwords na base de dados
    df.at[index,'dados_sem_stopwords'] = sem_stopwords
    #Aplicando a lematização
    lematizacao = lematization(sem_stopwords)
    #Armazenando os dados lematizados na base de dados
    df.at[index,'dados_lematizados'] = lematizacao

  return df

In [ ]:
df_treino=pre_processamento(df_treino)
df_teste=pre_processamento(df_teste)
df_treino

,index,alteracoes,target,dados_tokenizados,dados_sem_stopwords,dados_lematizados
479,479,painel acusando anomalia referente ao air bag ...,1.0,"[painel, acusando, anomalia, referente, ao, ai...","[painel, acusando, anomalia, referente, air, b...","[painel, acusar, anomalia, referente, air, bag..."
7020,7020,a vtr encontra com a base da antena esquerda d...,1.0,"[a, vtr, encontra, com, a, base, da, antena, e...","[vtr, encontra, base, antena, esquerda, danifi...","[vtr, encontrar, base, antena, esquerdo, danif..."
7368,7368,viatura retornou com a equipe que estava convo...,0.0,"[viatura, retornou, com, a, equipe, que, estav...","[viatura, retornou, equipe, convocada, sao, pa...","[viatura, retornar, equipe, convocar, Sao, Pau..."
4188,4188,01 liquido de arrefecimento do motor ok 02 flu...,0.0,"[liquido, de, arrefecimento, do, motor, ok, fl...","[liquido, arrefecimento, motor, ok, fluido, fr...","[liquido, arrefecimento, motor, ok, fluido, fr..."
6960,6960,em manutencao pane eletrica sem movimentacao,0.0,"[em, manutencao, pane, eletrica, sem, moviment...","[manutencao, pane, eletrica, sem, movimentacao]","[manutencao, pane, eletrico, sem, movimentacao]"
...,...,...,...,...,...,...
7374,7374,sem disco no tacografo,0.0,"[sem, disco, no, tacografo]","[sem, disco, tacografo]","[sem, disco, tacografo]"
4331,4331,camara de re nao funciona,1.0,"[camara, de, re, nao, funciona]","[camara, re, nao, funciona]","[Camara, Re, nao, funcionar]"
2304,2304,viatura envolvida em acidente de transito,1.0,"[viatura, envolvida, em, acidente, de, transito]","[viatura, envolvida, acidente, transito]","[viatura, envolver, acidente, transito]"
5265,5265,pastilhas ruins desgaste excessivo dos pneus d...,1.0,"[pastilhas, ruins, desgaste, excessivo, dos, p...","[pastilhas, ruins, desgaste, excessivo, pneus,...","[pastilha, ruim, desgaste, excessivo, pneu, di..."


#Gerando as representações

In [ ]:
#número de dimensões 
num_dimensoes = 200

In [ ]:
modelo_linguagem = gensim.models.Word2Vec(df_treino['dados_tokenizados'],sg=0,min_count=2,window=10,vector_size=num_dimensoes)#alterar parametros
#modelo_linguagem.wv.index_to_key32
print(type(modelo_linguagem))

<class 'gensim.models.word2vec.Word2Vec'>


In [ ]:
def gerar_representacao(texto, num_dimensoes, metodo):
  matrix = np.zeros((len(texto), num_dimensoes))

  for i in range(len(texto)):
    tokens = texto.iloc[i]
    matrix[i] = soma_vetores(tokens, num_dimensoes)
    if metodo == 'average' and len(tokens) > 0:
      matrix[i] = matrix[i]/len(tokens)

  return matrix


def soma_vetores(tokens, num_dimensoes):
  vetor_texto = np.zeros(num_dimensoes)
  for token in tokens:
    try:
      vetor_texto += modelo_linguagem.wv.get_vector(token)
    except KeyError:
      continue
  return vetor_texto

In [ ]:
repr_treino = gerar_representacao(df_treino['dados_tokenizados'], num_dimensoes, 'sum')
repr_treino

array([[  0.02714161,  -0.9708842 ,   0.38056402, ...,  -2.79084308,
         -0.88433548,  -0.99091762],
       [  1.46348798,  -4.91803079,   2.13733005, ..., -11.89681513,
         -4.0629909 ,  -3.78551131],
       [  0.6795584 ,  -2.71616162,   1.61262568, ...,  -7.38627539,
         -2.87154143,  -2.65503365],
       ...,
       [  0.65657083,  -1.13533255,   0.59337595, ...,  -2.44950657,
         -0.9203021 ,  -0.67286692],
       [  0.32150661,  -0.93793256,   0.36986144, ...,  -2.17069433,
         -0.60626535,  -0.55202072],
       [ -0.33345897,  -1.52685888,   1.24038257, ...,  -5.74915209,
         -2.40695097,  -2.39228422]])

#Construindo o modelo de Classificação

In [ ]:
classificador = SVC(kernel='linear',C=2.5)#alterar parametros
classificador.fit(repr_treino, df_treino['target'])

SVC(C=2.5, kernel='linear')

In [ ]:
repr_teste = gerar_representacao(df_teste['dados_tokenizados'], num_dimensoes, 'sum')#usar essa funcao para predizer resultado de uma frase so
predicoes = classificador.predict(repr_teste)
print(repr_teste)
print(predicoes)

[[ 2.30761270e-01 -1.77062949e+00 -4.97606280e-01 ... -3.48158351e+00
   2.18006605e-01 -4.43865387e-01]
 [-6.05754461e-03 -2.55826053e+00  2.02138644e+00 ... -8.80511187e+00
  -3.63213797e+00 -3.75429130e+00]
 [-1.81660748e-01 -3.97655443e+00  2.96504051e+00 ... -1.42943799e+01
  -6.00374537e+00 -6.45850298e+00]
 ...
 [-4.26685246e-01 -1.30726994e+00  1.47545817e+00 ... -5.47168024e+00
  -2.60853002e+00 -2.52235126e+00]
 [-8.41852558e-02 -1.64027369e+00  4.45269880e-01 ... -5.03862001e+00
  -1.32892365e+00 -1.68032483e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
[0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0.
 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0.
 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0.
 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0.

In [ ]:
#f1_score(df_teste['target'],predicoes,average='macro')
#classification_report(df_teste['target'], predicoes, digits=4)
# imprimir a acurácia do modelo
accu = accuracy_score(df_teste['target'], predicoes)
print(f"Acurácia: {accu:.2f}")

Acurácia: 0.75


In [ ]:

problemas=['pastilha de freio','marcha escapando','motor quebrado','o carro quebrou','FAROL AUXILIAR DIREITO QUEBRADO, LANTERNA TRASEIRA ESQUERDA QUEBRADA; PARA CHOQUE DIANTEIRO LADO DIREITO QUEBRADO. P  RA BRISA TRINCADO.','Barulho na parte traseira do veiculo, possivelmente alguma pe  a solta.', 'Encaminhado para oficina MF Auto Center (Ticketlog)', 'A viatura apresentou problemas na bomba de combustivel, durante deslocamento ao p  tio do DER( Aracariguama).','vtr com problemas de suspensao']
df1 =pd.DataFrame({'alteracoes': problemas})
#Remover as linhas nas quais os elementos sejam nan
df1.isnull().sum()

repr = gerar_representacao(df1['alteracoes'], num_dimensoes, 'sum')#usar essa funcao para predizer resultado de uma frase so
predicoes_1 = classificador.predict(repr)
print(df1)
#print(repr)
print(predicoes_1)


                                          alteracoes
0                                  pastilha de freio
1                                   marcha escapando
2                                     motor quebrado
3                                    o carro quebrou
4  FAROL AUXILIAR DIREITO QUEBRADO, LANTERNA TRAS...
5  Barulho na parte traseira do veiculo, possivel...
6  Encaminhado para oficina MF Auto Center (Ticke...
7  A viatura apresentou problemas na bomba de com...
8                     vtr com problemas de suspensao
[0. 0. 0. 0. 0. 0. 0. 0. 0.]


<h4>Plotando a matriz de confusão</h4>

In [ ]:
matriz = confusion_matrix(df_teste['target'], predicoes)
print(matriz)

print(f"\n\nVerdadeiro Positivo: {matriz[0][0]}")
print(f"Falso Positivo: {matriz[0][1]}")
print(f"Falso Negativo: {matriz[1][0]}")
print(f"Verdadeiro Negativo: {matriz[1][1]}")
#http://www2.decom.ufop.br/imobilis/svm-entendendo-sua-matematica-parte-1-a-margem/
#https://about.fb.com/news/2022/01/first-self-supervised-algorithm-for-speech-vision-text/

[[135  25]
 [ 51  91]]


Verdadeiro Positivo: 135
Falso Positivo: 25
Falso Negativo: 51
Verdadeiro Negativo: 91


#Aplicação de Orientação a Objetos

In [ ]:
from datetime import datetime
class alteracao:
  urgencia=False
  descricao=None
  dataModificacao=None

  def __init__(self,descricao,dataModificacao=None):
    self.descricao=descricao
    self.dataModificacao=dataModificacao
    if dataModificacao==None or dataModificacao.replace(' ','')=='' or descricao==None:
      self.dataModificacao=datetime.now().strftime("%H:%M:%S")
  def setUrgencia(self,urgencia):
    self.urgencia= True if urgencia==1 else False 
  def __str__(self):
    return {'urgencia':self.urgencia ,'descricao': self.descricao,'dataModificacao': self.dataModificacao}

class viatura:
  alteracoes=list()
  def __init__(self,placa,dataDesativacao=None,alteracoes=alteracao('conforme pdi anterior')):
    self.placa=placa
    self.alteracoes.append(alteracoes)
  def __str__(self):
    return {'placa':self.placa,'problemas':problemas}

class relatorio:
  baseDeDados=''
  def setBasedeDados():
    pass
  def gerarRelatorio():
    pass

#BACKUP

<h4>Armazenando os valores em um dataframe<h4>

In [ ]:
#Lendo a base de dados
df = pd.read_csv('data.csv')
#df.to_parquet('data.parquet')
#df = pd.read_parquet('data.parquet')

#Criando as novas colunas
df['dados_tokenizados'] = None 
df['dados_sem_stopwords'] = None
df['dados_lematizados'] = None

sentences=list()
#Interando sobre os dados e realizando o pré-processamento
for i, value in df.iterrows():
  #Aplicando a tokenização
  tokenizacao = tokenization(value['alteracoes'])
  #print(value['target'])
  #Armazenando os dados tokenizados na base de dados
  df.at[i,'dados_tokenizados'] = tokenizacao
  #Aplicando a remoção de stopwords
  sem_stopwords = removeStopwords(tokenizacao)
  #Armazenando os dados sem stopwords na base de dados
  df.at[i,'dados_sem_stopwords'] = sem_stopwords
  #Aplicando a lematização
  lematizacao = lematization(sem_stopwords)
  #Armazenando os dados lematizados na base de dados
  df.at[i,'dados_lematizados'] = lematizacao
  sentences.append(lematizacao)   
#for x in sentences:
model = Word2Vec(sentences, min_count=3)

keys=model.wv.index_to_key
#print(keys)
vetores=model.wv
print(vetores)


KeyedVectors<vector_size=100, 799 keys>


In [ ]:
#Salvando em formato Parquet
df.to_parquet('data.parquet')
#dataframe=pd.read_parquet('data.parquet')
#print(dataframe)
#exibir o dataframe
df['alteracoes']

0                                                      nc
1                                   conforme pdi anterior
2        painel informando a necessidade de troca de oleo
3                                                      sa
4                                 sem alteracoes visiveis
                              ...                        
7558         vtr encaminhada para troca de oleo e filtros
7559                            vidros traseiro nao abrem
7560                                     idem pd anterior
7561    km diferente da pdi viatura levada para a conc...
7562    vidros travando para subir farol direito queim...
Name: alteracoes, Length: 7563, dtype: object

Esta frase é de caráter:negativo
Precisão da previsão:0.8571428571428571


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<h4>Backup</h4>

In [ ]:
# muito ruim , accuracia ruim e demora pra calcular e o tempo de execucução cresce exponencialmente
from textblob.classifiers import NaiveBayesClassifier
from textblob import TextBlob
import nltk
import pandas as pd
import datetime

nltk.download('punkt')

#Lendo a base de dados
df = pd.read_csv('data.csv')
df.to_parquet('data.parquet')
df=pd.read_parquet('data.parquet')
#df.to_parquet('data.parquet')
#df = pd.read_parquet('data.parquet')

#Criando as novas colunas
df['dados_tokenizados'] = None 
df['dados_sem_stopwords'] = None
df['dados_lematizados'] = None

train_set=list()
test_set=list()
sentences=list()
start_index=2000
train_limit=3000
test_limit=train_limit+1000
#Interando sobre os dados e realizando o pré-processamento
for i, value in df.iterrows():
  #print(i)
  if i<start_index:
    continue
  elif i>=start_index and i<=train_limit:
    train_set.append((value['alteracoes'],(value['target'])))
  elif i>train_limit and i<test_limit :
    test_set.append((value['alteracoes'],(value['target'])))
  else:
    break
print('')
cl = NaiveBayesClassifier(train_set)
accuracy=1
#accuracy = cl.accuracy(test_set[0:(len(test_set)-1)//2])
print('classification done')
#teste
frases = ['vidros traseiro nao abrem','sem problema','precisa de conserto','precisa trocar os pneus','quebrado','em perfeitas condicoes','defeito','arranhada','falhas','ineficiente','embreagem precisa de reparo']
#for i,frase in enumerate(frases):
#  blob = TextBlob(frase,classifier=cl)
#  print(f'{i}|{frase}')
#  print('Esta frase é de caráter:{}'.format(blob.classify()))
#print('Precisão da previsão:{}'.format(accuracy))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



classification done


In [ ]:
frase=frases[5]
blob = TextBlob(frase,classifier=cl)
print(f'{i}|{frase}')
print('Esta frase é de caráter:{}'.format(blob.classify()))
print('Precisão da previsão:{}'.format(accuracy))

4000|em perfeitas condicoes


KeyboardInterrupt: ignored

In [ ]:
# codigo legado
from textblob.classifiers import NaiveBayesClassifier
from textblob import TextBlob

nltk.download('punkt')



train_set = [
    ('Eu adoro comer hamburguer', 'positivo'),
    ('Este lugar é horrível', 'negativo'),
    ('Você é uma pessoa adorável', 'positivo'),
    ('Você é uma pessoa horrível', 'negativo'),
    ('A festa está ótima', 'positivo'),
    ('A festa está péssima', 'negativo'),
    ('Este lugar é maravilhoso', 'positivo'),
    ('Estou cansado disso.', 'negativo'),
    ('Eu te odeio', 'negativo'),
    ('Eu te adoro', 'positivo'),
    ('Eu te amo', 'positivo'),
    ('Você é incrível','positivo'),
    ('Eu estou com muita raiva','negativo'),
    ('Eu odeio essa linguagem','negativo'),
    ('Essa linguagem é fantátisca','positivo'),
    ('Essa linguagem é muito boa','positivo'),
    ('Que comida gostosa','positivo'),
    ('Que comida horrível','negativo'),
    ('Estou me sentindo ótimo','positivo'),
    ('Hoje eu estou péssimo','negativo'),
    ('Ele merece ser deportado','negativo'),
    ('Ele obteve um premio,um trofeu','positivo'),
    ('ele merece um cafe','positivo'),
    ('ele merece um tapa na cara','negativo'),
    ('Ele obteve um trofeu de asno','negativo'),
    ('Ele é um asno','negativo')
]
test_set = [
    ('Ótima linguagem', 'positivo'),
    ('Péssima essa linguagem', 'negativo'),
    ('Você é horrível', 'negativo'),
    ('Comida gostosa!', 'positivo'),
    ('Que raiva!', 'negativo'),
    ('Ótima festa!', 'positivo'),
    ('Eu não odeio todo mundo','positivo')
]

cl = NaiveBayesClassifier(train_set)
accuracy = cl.accuracy(test_set)

frase = 'Rativa merece um trofeu'

blob = TextBlob(frase,classifier=cl)
print('Esta frase é de caráter:{}'.format(blob.classify()))
print('Precisão da previsão:{}'.format(accuracy))

#Verificando a distribuição dos dados

In [ ]:
import pandas as pd
import seaborn as sns

df = pd.read_csv('data.csv')
new_data = df.dropna(axis = 0, how ='any') 

df.head()

sns.pairplot(df[['target']], hue='target')